<a href="https://colab.research.google.com/github/manuel-suarez/super_resolution_cnn/blob/main/Seminario_de_Tesis_III_SRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Super-Resolution Convolutional Neural Network for Image Restoration
Tutorial for: [S-RCNN for SRI](https://medium.datadriveninvestor.com/using-the-super-resolution-convolutional-neural-network-for-image-restoration-ff1e8420d846)

## 1.- Importing Packages

In [1]:
# Check package versions
import sys
import keras
import cv2
import numpy
import matplotlib
import skimage

print('Python: {}'.format(sys.version))
print('Keras: {}'.format(keras.__version__))
print('OpenCV: {}'.format(cv2.__version__))
print('NumPy: {}'.format(numpy.__version__))
print('Matplotlib: {}'.format(matplotlib.__version__))
print('Scikit-Image: {}'.format(skimage.__version__))

Python: 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
Keras: 2.8.0
OpenCV: 4.6.0
NumPy: 1.21.6
Matplotlib: 3.2.2
Scikit-Image: 0.18.3


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from skimage.metrics import structural_similarity as ssim
from matplotlib import pyplot as plt
import cv2
import numpy as np
import math
import os

# python magic function, displays
%matplotlib inline

## 2.- Image Quality Metrics

In [5]:
# define a function for peak signal-to-noise ratio (PSNR)
def psnr(target, ref):
  # assume RGB image
  target_data = target.astype(float)
  ref_data = ref.astype(float)

  diff = ref_data - target_data
  diff = diff.flatten('C')

  rmse = math.sqrt(np.mean(diff ** 2.))

  return 20 * math.log10(255. / rmse)

# define function for mean squared error (MSE)
def mse(target, ref):
  # the MSE between the two images is the sum of the squared difference between the two images
  err = np.sum((target.astype('float') - ref.astype('float')) ** 2)
  err /= float(target.shape[0] * target.shape[1])

  return err

# define function that combines all three image quality metrics
def compare_images(target, ref):
  scores = []
  scores.append(psnr(target, ref))
  scores.append(mse(target, ref))
  scores.append(ssim(target, ref, multichannel=True))

  return scores

## 3.- Preparing images

In [20]:
# prepare degraded images by introducing quality distortions via resizing
def prepare_images(path, factor):
  # loop through the files in the directory
  for file in os.listdir(path):
    # open the file
    img = cv2.imread(path + '/' + file)

    # find old and new image dimensions
    h, w, _ = img.shape
    new_height = int(h / factor)
    new_width = int(w / factor)
    dim = (new_width, new_height)

    # resize the image - down
    img = cv2.resize(img, dim, interpolation = cv2.INTER_LINEAR)

    # resize the image - up
    img = cv2.resize(img, (w, h), interpolation = cv2.INTER_LINEAR)

    # save the image
    print('Saving {}'.format(file))
    cv2.imwrite('images/{}'.format(file), img)

prepare_images('source/', 2)

Saving bird_GT.bmp
Saving face.bmp
Saving lenna.bmp
Saving coastguard.bmp
Saving baboon.bmp
Saving pepper.bmp
Saving zebra.bmp
Saving bridge.bmp
Saving ppt3.bmp
Saving flowers.bmp
Saving monarch.bmp
Saving butterfly_GT.bmp
Saving barbara.bmp
Saving baby_GT.bmp
Saving man.bmp
Saving comic.bmp
Saving woman_GT.bmp
Saving head_GT.bmp
Saving foreman.bmp


## 3.- Testing Low-Resolution Images

In [22]:
# test the generated images using the image quality metrics
for file in os.listdir('images/'):
  # open target and reference images
  target = cv2.imread('images/{}'.format(file))
  ref = cv2.imread('source/{}'.format(file))

  # calculate score
  scores = compare_images(target, ref)

  # print all three scores with new line characters (\n)
  print('{}\nPSN: {}\nMSE: {}\nSSIM: {}\n'.format(file, scores[0], scores[1], scores[2]))

bird_GT.bmp
PSN: 32.896644728720005
MSE: 100.12375819830247
SSIM: 0.9533644866026473

face.bmp
PSN: 30.99220650287191
MSE: 155.23189718546524
SSIM: 0.8008439492289884

lenna.bmp
PSN: 31.47349297867539
MSE: 138.94800567626953
SSIM: 0.8460989200521499

coastguard.bmp
PSN: 27.161600663887082
MSE: 375.00887784090907
SSIM: 0.756950063354931

baboon.bmp
PSN: 22.157084083442548
MSE: 1187.1161333333334
SSIM: 0.629277587900277

pepper.bmp
PSN: 29.88947161686106
MSE: 200.1033935546875
SSIM: 0.8357937568464359

zebra.bmp
PSN: 27.909840639329513
MSE: 315.6585459528818
SSIM: 0.8911656209329116

bridge.bmp
PSN: 25.850528790115554
MSE: 507.1643714904785
SSIM: 0.7804245912255268

ppt3.bmp
PSN: 24.84926168950471
MSE: 638.6684263912582
SSIM: 0.9284023942315316

flowers.bmp
PSN: 27.454504805386144
MSE: 350.55093922651935
SSIM: 0.8697286286974628

monarch.bmp
PSN: 30.196242365288896
MSE: 186.45643615722656
SSIM: 0.9439574293434104

butterfly_GT.bmp
PSN: 24.782076560337416
MSE: 648.6254119873047
SSIM: 0.87

## 4.- Building the SRCNN Model

In [23]:
# define the SRCNN model
def model():
  # define model type
  SRCNN = Sequential()

  # add model layers
  SRCNN.add(Conv2D(filters=128, 
                   kernel_size=(9,9), 
                   kernel_initializer='glorot_uniform',
                   activation='relu',
                   padding='valid',
                   use_bias=True,
                   input_shape=(None, None, 1)))
  SRCNN.add(Conv2D(filters=64,
                   kernel_size=(3,3),
                   kernel_initializer='glorot_uniform',
                   activation='relu',
                   padding='same',
                   use_bias=True))
  SRCNN.add(Conv2D(filters=1,
                   kernel_size=(5,5),
                   kernel_initializer='glorot_uniform',
                   activation='linear',
                   padding='valid',
                   use_bias=True))
  
  # define optimizer
  adam = Adam(lr=0.0003)

  # compile model
  SRCNN.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])

  return SRCNN

## 5.- Deploying the SRCNN